In [1]:
from bs4 import BeautifulSoup
import requests
import shutil
from detect import detect
import torch
import cv2
from PIL import Image
import os
import io
import pandas as pd
from flask import Flask,Response,render_template,request,redirect

In [2]:
def find_result(img,img_sz):
    print("checking",img)
    opt = {"agnostic_nms":False, "augment":False, 
           "classes":None, "conf_thres":0.1, 
           "device":'', "exist_ok":False, 
           "img_size":img_sz, "iou_thres":0.45, 
           "name":'exp', "no_trace":True, "nosave":False, 
           "project":'runs/detect', "save_conf":True, 
           "save_txt":True, "source":img, "update":False, 
           "view_img":False, "weights":['yolov7.pt']}
    with torch.no_grad():
        res = detect(opt)
    return res

In [3]:
def directory_handle(URL,ID,re,source):
    print("dir han")
    for i in re:
        if (os.path.exists(path+i)==False):
            os.mkdir(path+i)

        try:
            shutil.copy(source, path+i)
            imna=path+i+(source.split("/")[-1])
            csv_update(URL,ID,imna,re)
            print("Copied")
        except:
            print("Failed to copy")

In [4]:
def csv_update(URL,ID,imna,re):
    df=pd.DataFrame({"URL":[URL,],"ID":[ID,],"img_name":[imna,],"result":[re,]})
    df.to_csv(path+"urlid.csv",mode='a',index=False,header=False)
    print("csv up")

In [5]:
def getdata(url):
    r = requests.get(url)
    return r.text

In [6]:
def scrape_image(URL):
    htmldata = getdata(URL) 
    soup = BeautifulSoup(htmldata, 'html.parser')
    src=[]
    images = soup.find_all('img')

    for item in images:
        try:
            if len(item['src'])>0:
                src.append(item['src'])
        except:
            pass
    print(src)
#     print(images)
    return src

In [7]:
def image_downoad(src):
    j=0
    path=app.root_path+"\\static\\"
    sr_path= path +"Src_image\\"
    urlid= pd.read_csv(path+"urlid.csv")
    if URL not in urlid["URL"]:
        ID=urlid["ID"].iloc[-1]+1
    else:
        return
        
    for i in src:
        try:
            r = requests.get(i, stream=True)
            if r.status_code == 200:
                img_path =os.path.join(sr_path,str(ID)+"_"+str(j)+".jpg")
                with open(img_path, 'wb') as f: 
                    r.raw.decode_content = True
                    shutil.copyfileobj(r.raw, f)
                img_sz=img_size#(im.shape[0]+im.shape[1])*3
                res = find_result(img_path,img_sz)

                if len(res.keys())!=0:
                    re= res.keys()
                    directory_handle(URL,ID,re,img_path)

                j+=1
                
        except Exception as e:
            print(e)
            j+=1
            continue

In [9]:
URL=None

In [10]:
app = Flask(__name__)

In [11]:
min_size=10
img_size=224
path=app.root_path+"\\static\\"
sr_path=path#'''+"Src_image\\"'''
print(path)
if not (os.path.exists(sr_path)):
    print(sr_path)
    os.makedirs(sr_path)
print("path to ",os.path.exists(sr_path))

C:\Users\sreea\yolo_model\yolov7\static\
path to  True


In [12]:
@app.route('/', methods =["GET", "POST"])
def i_url():
    global URL
    if request.method == "POST":
        URL = request.form.get("url")
        print(URL)
        src=scrape_image(URL)
        image_downoad(src)
        return redirect('/image')
    return render_template("main.html")

In [13]:
@app.route('/')
def index():
    return render_template('main.html')

In [14]:
@app.route('/image')
def image():
    image_names = [sr_path+"\\Src_image\\"+i for i in os.listdir(sr_path+"\\Src_image\\")]
    print(image_names)
    return render_template('image.html',images=image_names)

In [15]:
if __name__=='__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Jan/2023 20:16:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2023 20:16:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2023 20:16:40] "GET /image HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2023 20:16:40] "GET /static/C:%5CUsers%5Csreea%5Cyolo_model%5Cyolov7%5Cstatic%5C%5CSrc_image%5C1_0.jpg HTTP/1.1" 404 -
127.0.0.1 - - [26/Jan/2023 20:16:40] "GET /static/C:%5CUsers%5Csreea%5Cyolo_model%5Cyolov7%5Cstatic%5C%5CSrc_image%5C1_1.jpg HTTP/1.1" 404 -
127.0.0.1 - - [26/Jan/2023 20:16:40] "GET /static/C:%5CUsers%5Csreea%5Cyolo_model%5Cyolov7%5Cstatic%5C%5CSrc_image%5C1_10.jpg HTTP/1.1" 404 -
127.0.0.1 - - [26/Jan/2023 20:16:40] "GET /static/C:%5CUsers%5Csreea%5Cyolo_model%5Cyolov7%5Cstatic%5C%5CSrc_image%5C1_11.jpg HTTP/1.1" 404 -
127.0.0.1 - - [26/Jan/2023 20:16:40] "GET /static/C:%5CUsers%5Csreea%5Cyolo_model%5Cyolov7%5Cstatic%5C%5CSrc_image%5C1_12.jpg HTTP/1.1" 404 -
127.0.0.1 - - [26/Jan/2023 20:16:40] "GET /stat

['C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_0.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_1.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_10.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_11.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_12.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_13.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_14.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_15.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_16.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_17.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_18.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_2.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_3.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_4.jpg', 'C:\\Users

YOLOR  v0.1-115-g072f76c torch 1.12.1+cpu CPU



checking C:\Users\sreea\yolo_model\yolov7\static\Src_image\2_0.jpg
Fusing layers... 


...

YOLOR  v0.1-115-g072f76c torch 1.12.1+cpu CPU



checking C:\Users\sreea\yolo_model\yolov7\static\Src_image\2_10.jpg
Fusing layers... 


Model Summary: 306 layers, 36905341 parameters, 6652669 gradients
127.0.0.1 - - [26/Jan/2023 20:16:58] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [26/Jan/2023 20:16:58] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [26/Jan/2023 20:16:58] "GET /image HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2023 20:16:58] "GET /image HTTP/1.1" 200 -


RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
Image Not Found C:\Users\sreea\yolo_model\yolov7\static\Src_image\2_10.jpg
['C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_0.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_1.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_10.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_11.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_12.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_13.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_14.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_15.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_16.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_17.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_18.jpg', 'C:\\Users\\sreea\\yolo_model\\yolov7\\static\\\\Src_image\\1_2.jpg

127.0.0.1 - - [26/Jan/2023 20:16:58] "GET /static/C:%5CUsers%5Csreea%5Cyolo_model%5Cyolov7%5Cstatic%5C%5CSrc_image%5C1_0.jpg HTTP/1.1" 404 -
127.0.0.1 - - [26/Jan/2023 20:16:58] "GET /static/C:%5CUsers%5Csreea%5Cyolo_model%5Cyolov7%5Cstatic%5C%5CSrc_image%5C1_0.jpg HTTP/1.1" 404 -
127.0.0.1 - - [26/Jan/2023 20:16:58] "GET /static/C:%5CUsers%5Csreea%5Cyolo_model%5Cyolov7%5Cstatic%5C%5CSrc_image%5C1_1.jpg HTTP/1.1" 404 -
127.0.0.1 - - [26/Jan/2023 20:16:58] "GET /static/C:%5CUsers%5Csreea%5Cyolo_model%5Cyolov7%5Cstatic%5C%5CSrc_image%5C1_1.jpg HTTP/1.1" 404 -
127.0.0.1 - - [26/Jan/2023 20:16:58] "GET /static/C:%5CUsers%5Csreea%5Cyolo_model%5Cyolov7%5Cstatic%5C%5CSrc_image%5C1_10.jpg HTTP/1.1" 404 -
127.0.0.1 - - [26/Jan/2023 20:16:58] "GET /static/C:%5CUsers%5Csreea%5Cyolo_model%5Cyolov7%5Cstatic%5C%5CSrc_image%5C1_10.jpg HTTP/1.1" 404 -
127.0.0.1 - - [26/Jan/2023 20:16:58] "GET /static/C:%5CUsers%5Csreea%5Cyolo_model%5Cyolov7%5Cstatic%5C%5CSrc_image%5C1_11.jpg HTTP/1.1" 404 -
127.0.0.1 

In [18]:
image_names = [i for i in os.listdir(sr_path+"\\Src_image\\")]
print(image_names)

['1_0.jpg', '1_1.jpg', '1_10.jpg', '1_11.jpg', '1_12.jpg', '1_13.jpg', '1_14.jpg', '1_15.jpg', '1_16.jpg', '1_17.jpg', '1_18.jpg', '1_2.jpg', '1_3.jpg', '1_4.jpg', '1_5.jpg', '1_6.jpg', '1_7.jpg', '1_8.jpg', '1_9.jpg', '2_0.jpg', '2_1.jpg', '2_10.jpg', '2_2.jpg', '2_3.jpg', '2_4.jpg', '2_5.jpg', '2_6.jpg', '2_7.jpg', '2_8.jpg', '2_9.jpg']
